**DEPLOYING THE MODEL**

---



SAVE AND LOAD THE MODEL TO USE IT AS SAVED MODEL

In [ ]:
import tensorflow as tf
!pip install tflite-runtime

     |████████████████████████████████| 2.2 MB 5.6 MB/s 


In [ ]:
#tf.saved_model.save(model, path_to_dir) #SAVING THE MODEL
!cd /content/drive/MyDrive/SDP/saved_model
model = tf.saved_model.load ('/content/drive/MyDrive/SDP/saved_model')

CONVERTING MODEL FROM TENSORFLOW TO TENSORFLOW LITE

In [ ]:
class Model(tf.Module):

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
  def encode(self, x):
    result = tf.strings.as_string(x)
    return {
         "encoded_result": result
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
  def decode(self, x):
    result = tf.strings.to_number(x)
    return {
         "decoded_result": result
    }

In [ ]:
model = Model()

# Save the model
SAVED_MODEL_PATH = '/content/drive/MyDrive/SDP/saved_model'

tf.saved_model.save(
    model, SAVED_MODEL_PATH,
    signatures={
      'encode': model.encode.get_concrete_function(),
      'decode': model.decode.get_concrete_function()
    })

# Convert the saved model using TFLiteConverter
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/SDP/saved_model/assets


{'decode': {'inputs': ['x'], 'outputs': ['decoded_result']}, 'encode': {'inputs': ['x'], 'outputs': ['encoded_result']}}


In [ ]:
# Save the model.
with open('Unmask_the_masked.tflite', 'wb') as f:
  f.write(tflite_model)